In [1]:
import tensorflow as tf
import numpy as np
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype(np.float32).reshape(-1, 28*28) / 255.0
x_test = x_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
x_valid, x_train = x_train[:5000], x_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [2]:
def shuffle_batch(x, y, batch_size):
    rnd_idx = np.random.permutation(len(x))
    n_batches = len(x) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        x_batch, y_batch = x[batch_idx], y[batch_idx]
        yield x_batch, y_batch

In [3]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
tf.set_random_seed(0)
np.random.seed(0)

In [4]:
lr = 0.01
epoch = 20
batch_size = 200
threshold = 1.0

In [5]:
tf.get_default_graph()

In [6]:
tf.get_default_graph().get_operations()

[]

In [7]:
for op in tf.get_default_graph().get_operations():
    print(op)

In [8]:
saver = tf.train.import_meta_graph("./save/my_model_final.ckpt.meta")

In [9]:
saver

In [10]:
tf.get_default_graph()

In [11]:
tf.get_default_graph().get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'y' type=Placeholder>,
 <tf.Operation 'hidden1/kernel/Initializer/random_uniform/shape' type=Const>,
 <tf.Operation 'hidden1/kernel/Initializer/random_uniform/min' type=Const>,
 <tf.Operation 'hidden1/kernel/Initializer/random_uniform/max' type=Const>,
 <tf.Operation 'hidden1/kernel/Initializer/random_uniform/RandomUniform' type=RandomUniform>,
 <tf.Operation 'hidden1/kernel/Initializer/random_uniform/sub' type=Sub>,
 <tf.Operation 'hidden1/kernel/Initializer/random_uniform/mul' type=Mul>,
 <tf.Operation 'hidden1/kernel/Initializer/random_uniform' type=Add>,
 <tf.Operation 'hidden1/kernel' type=VariableV2>,
 <tf.Operation 'hidden1/kernel/Assign' type=Assign>,
 <tf.Operation 'hidden1/kernel/read' type=Identity>,
 <tf.Operation 'hidden1/bias/Initializer/zeros' type=Const>,
 <tf.Operation 'hidden1/bias' type=VariableV2>,
 <tf.Operation 'hidden1/bias/Assign' type=Assign>,
 <tf.Operation 'hidden1/bias/read' type=Identity>,
 <tf.Operation 

In [12]:
for op in tf.get_default_graph().get_operations():
    print(op)

name: "x"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
      dim {
        size: 784
      }
    }
  }
}

name: "y"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "shape"
  value {
    shape {
      unknown_rank: true
    }
  }
}

name: "hidden1/kernel/Initializer/random_uniform/shape"
op: "Const"
attr {
  key: "_class"
  value {
    list {
      s: "loc:@hidden1/kernel"
    }
  }
}
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_INT32
      tensor_shape {
        dim {
          size: 2
        }
      }
      tensor_content: "\020\003\000\000,\001\000\000"
    }
  }
}

name: "hidden1/kernel/Initializer/random_uniform/min"
op: "Const"
attr {
  key: "_class"
  value {
    list {
      s: "loc:@hidden1/kernel"
    }
  }
}
attr {
  key: "dtype"
  value {
    type: DT_F

In [13]:
for op in tf.get_default_graph().get_operations():
    print(op.name)

x
y
hidden1/kernel/Initializer/random_uniform/shape
hidden1/kernel/Initializer/random_uniform/min
hidden1/kernel/Initializer/random_uniform/max
hidden1/kernel/Initializer/random_uniform/RandomUniform
hidden1/kernel/Initializer/random_uniform/sub
hidden1/kernel/Initializer/random_uniform/mul
hidden1/kernel/Initializer/random_uniform
hidden1/kernel
hidden1/kernel/Assign
hidden1/kernel/read
hidden1/bias/Initializer/zeros
hidden1/bias
hidden1/bias/Assign
hidden1/bias/read
dnn/hidden1/MatMul
dnn/hidden1/BiasAdd
dnn/hidden1/Relu
hidden2/kernel/Initializer/random_uniform/shape
hidden2/kernel/Initializer/random_uniform/min
hidden2/kernel/Initializer/random_uniform/max
hidden2/kernel/Initializer/random_uniform/RandomUniform
hidden2/kernel/Initializer/random_uniform/sub
hidden2/kernel/Initializer/random_uniform/mul
hidden2/kernel/Initializer/random_uniform
hidden2/kernel
hidden2/kernel/Assign
hidden2/kernel/read
hidden2/bias/Initializer/zeros
hidden2/bias
hidden2/bias/Assign
hidden2/bias/read
dn

In [14]:
n_hidden4 = 20  # new layer
n_outputs = 10  # new layer

x = tf.get_default_graph().get_tensor_by_name("x:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

hidden3 = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/Relu:0")

new_hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="new_hidden4")
new_logits = tf.layers.dense(new_hidden4, n_outputs, name="new_outputs")

with tf.name_scope("new_loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=new_logits)
    loss = tf.reduce_mean(entropy, name="loss")

with tf.name_scope("new_eval"):
    correct = tf.nn.in_top_k(new_logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("new_train"):
    opt = tf.train.GradientDescentOptimizer(lr)
    train = opt.minimize(loss)

init = tf.global_variables_initializer()
new_saver = tf.train.Saver()

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [15]:
with tf.Session() as sess:
    init.run()
    saver.restore(sess, "./save/my_model_final.ckpt")

    for i in range(epoch):
        for x_batch, y_batch in shuffle_batch(x_train, y_train, batch_size):
            sess.run(train, feed_dict={x: x_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={x: x_valid, y: y_valid})
        print(i, "Validation accuracy:", accuracy_val)

    save_path = new_saver.save(sess, "./save/my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./save/my_model_final.ckpt
0 Validation accuracy: 0.9258
1 Validation accuracy: 0.9402
2 Validation accuracy: 0.9472
3 Validation accuracy: 0.951
4 Validation accuracy: 0.9544
5 Validation accuracy: 0.9564
6 Validation accuracy: 0.959
7 Validation accuracy: 0.9606
8 Validation accuracy: 0.9622
9 Validation accuracy: 0.9628
10 Validation accuracy: 0.9644
11 Validation accuracy: 0.9632
12 Validation accuracy: 0.9628
13 Validation accuracy: 0.9644
14 Validation accuracy: 0.966
15 Validation accuracy: 0.9656
16 Validation accuracy: 0.9664
17 Validation accuracy: 0.9678
18 Validation accuracy: 0.968
19 Validation accuracy: 0.9676
